In [17]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV

In [18]:
from google.colab import files
uploaded = files.upload()


Saving churn.csv to churn.csv


In [19]:
df = pd.read_csv('churn.csv')
df = pd.DataFrame(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [20]:
# df.duplicated('CustomerId').sum()
# Setting index to customerid column -->
df = df.set_index('CustomerId')
df['Geography'].value_counts()
df.replace({'Geography':{'France':2, 'Germany':1, 'Spain':0}}, inplace=True)
df['Geography'].value_counts()
df.replace({'Gender' : {'Male':0, 'Female':1}}, inplace=True)
df['NumOfProducts'].value_counts()
df.replace({'NumOfProducts':{1:0, 2:1, 3:1, 4:1}}, inplace=True)
# df.info()
df['Zero Balance'] = np.where(df['Balance']>0,1,0)

df.groupby(['Exited', 'Geography'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 15634602 to 15628319
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   Surname          10000 non-null  object 
 2   CreditScore      10000 non-null  int64  
 3   Geography        10000 non-null  int64  
 4   Gender           10000 non-null  int64  
 5   Age              10000 non-null  int64  
 6   Tenure           10000 non-null  int64  
 7   Balance          10000 non-null  float64
 8   NumOfProducts    10000 non-null  int64  
 9   HasCrCard        10000 non-null  int64  
 10  IsActiveMember   10000 non-null  int64  
 11  EstimatedSalary  10000 non-null  float64
 12  Exited           10000 non-null  int64  
 13  Zero Balance     10000 non-null  int64  
dtypes: float64(2), int64(11), object(1)
memory usage: 1.1+ MB


<ipython-input-20-e677d32f7ec3>:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({'Geography':{'France':2, 'Germany':1, 'Spain':0}}, inplace=True)
<ipython-input-20-e677d32f7ec3>:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({'Gender' : {'Male':0, 'Female':1}}, inplace=True)


In [21]:
X = df.drop(['Surname', 'Exited'], axis=1)
Y = df['Exited']
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 15634602 to 15628319
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CreditScore      10000 non-null  int64  
 2   Geography        10000 non-null  int64  
 3   Gender           10000 non-null  int64  
 4   Age              10000 non-null  int64  
 5   Tenure           10000 non-null  int64  
 6   Balance          10000 non-null  float64
 7   NumOfProducts    10000 non-null  int64  
 8   HasCrCard        10000 non-null  int64  
 9   IsActiveMember   10000 non-null  int64  
 10  EstimatedSalary  10000 non-null  float64
 11  Zero Balance     10000 non-null  int64  
dtypes: float64(2), int64(10)
memory usage: 1015.6 KB


In [22]:
rus = RandomUnderSampler(random_state= 2529)
x_rus, y_rus = rus.fit_resample(X,Y)
x_rus.shape, y_rus.shape, X.shape , Y.shape
Y.value_counts()
y_rus.value_counts()

ros = RandomOverSampler(random_state = 2529)
x_ros, y_ros = ros.fit_resample(X, Y)

x_ros.shape, y_ros.shape, X.shape, Y.shape
y_ros.value_counts()


X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size=0.3, random_state=2529 )

X_train_rus, X_test_rus, Y_train_rus, Y_test_rus = train_test_split(x_rus, y_rus, test_size=0.3 ,  random_state=420)
X_train_ros, X_test_ros, Y_train_ros, Y_test_ros = train_test_split(x_ros, y_ros, test_size=0.3,  random_state=420)


In [23]:
sc = StandardScaler()
X_train[['CreditScore', 'Age','Tenure', 'Balance', 'EstimatedSalary']] = sc.fit_transform(X_train[['CreditScore', 'Age','Tenure', 'Balance', 'EstimatedSalary']])
X_test[['CreditScore', 'Age','Tenure', 'Balance', 'EstimatedSalary']] = sc.fit_transform(X_test[['CreditScore', 'Age','Tenure', 'Balance', 'EstimatedSalary']])
X_train_ros[['CreditScore', 'Age','Tenure', 'Balance', 'EstimatedSalary']] = sc.fit_transform(X_train_ros[['CreditScore', 'Age','Tenure', 'Balance', 'EstimatedSalary']])
X_test_ros[['CreditScore', 'Age','Tenure', 'Balance', 'EstimatedSalary']] = sc.fit_transform(X_test_ros[['CreditScore', 'Age','Tenure', 'Balance', 'EstimatedSalary']])
X_train_rus[['CreditScore', 'Age','Tenure', 'Balance', 'EstimatedSalary']] = sc.fit_transform(X_train_rus[['CreditScore', 'Age','Tenure', 'Balance', 'EstimatedSalary']])
X_test_rus[['CreditScore', 'Age','Tenure', 'Balance', 'EstimatedSalary']] = sc.fit_transform(X_test_rus[['CreditScore', 'Age','Tenure', 'Balance', 'EstimatedSalary']])



svc = SVC()
svc.fit(X_train, Y_train)
y_pred = svc.predict(X_test)
confusion_matrix(Y_test, y_pred)
print(classification_report(Y_test, y_pred))

svc_rus = SVC()
svc_rus.fit(X_train_rus, Y_train_rus)
y_pred_rus = svc_rus.predict(X_test_rus)

confusion_matrix(Y_test_rus, y_pred_rus)
print(classification_report(Y_test_rus, y_pred_rus))

svc_ros = SVC()
svc_ros.fit(X_train_ros, Y_train_ros)
y_pred_ros = svc_ros.predict(X_test_ros)

confusion_matrix(Y_test_ros, y_pred_ros)
print(classification_report(Y_test_ros, y_pred_ros))


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

           0       0.80      1.00      0.89      2414
           1       0.00      0.00      0.00       586

    accuracy                           0.80      3000
   macro avg       0.40      0.50      0.45      3000
weighted avg       0.65      0.80      0.72      3000

              precision    recall  f1-score   support

           0       0.51      0.64      0.57       605
           1       0.54      0.40      0.46       618

    accuracy                           0.52      1223
   macro avg       0.52      0.52      0.52      1223
weighted avg       0.52      0.52      0.51      1223

              precision    recall  f1-score   support

           0       0.50      0.51      0.50      2409
           1       0.50      0.49      0.49      2369

    accuracy                           0.50      4778
   macro avg       0.50      0.50      0.50      4778
weighted avg       0.50      0.50      0.50      4778



In [24]:
grid_lin = SVC()

param_grid = {'C' : [0.1, 1, 10], 'gamma':[1,0.1,0.01], 'kernel':['rbf'],  'class_weight' :['balanced']}

grid_lin = GridSearchCV(SVC(), param_grid, refit = True, verbose = 2, cv = 2)
grid_lin.fit(X_train,Y_train)

grid_pred_lin = grid_lin.predict(X_test)
# print(classification_report(Y_test, grid_pred_lin))




grid_rus = SVC()

param_grid = {'C' : [0.1, 1, 10], 'gamma':[1,0.1,0.01], 'kernel':['rbf'],  'class_weight' :['balanced']}

grid_rus = GridSearchCV(SVC(), param_grid, refit = True, verbose = 2, cv = 2)
grid_rus.fit(X_train_rus,Y_train_rus)

grid_pred_rus = grid_rus.predict(X_test_rus)
# print(classification_report(Y_test_rus, grid_pred_rus))





grid_ros = SVC()

param_grid = {'C' : [0.1, 1, 10], 'gamma':[1,0.1,0.01], 'kernel':['rbf'],  'class_weight' :['balanced']}

grid_ros = GridSearchCV(SVC(), param_grid, refit = True, verbose = 2, cv = 2)
grid_ros.fit(X_train_ros,Y_train_ros)

grid_pred_ros = grid_ros.predict(X_test_ros)
# print(classification_report(Y_test_ros, grid_pred_ros))

Fitting 2 folds for each of 9 candidates, totalling 18 fits
[CV] END ..C=0.1, class_weight=balanced, gamma=1, kernel=rbf; total time=   1.6s
[CV] END ..C=0.1, class_weight=balanced, gamma=1, kernel=rbf; total time=   2.5s
[CV] END C=0.1, class_weight=balanced, gamma=0.1, kernel=rbf; total time=   2.2s
[CV] END C=0.1, class_weight=balanced, gamma=0.1, kernel=rbf; total time=   1.6s
[CV] END C=0.1, class_weight=balanced, gamma=0.01, kernel=rbf; total time=   1.7s
[CV] END C=0.1, class_weight=balanced, gamma=0.01, kernel=rbf; total time=   2.2s
[CV] END ....C=1, class_weight=balanced, gamma=1, kernel=rbf; total time=   2.6s
[CV] END ....C=1, class_weight=balanced, gamma=1, kernel=rbf; total time=   2.9s
[CV] END ..C=1, class_weight=balanced, gamma=0.1, kernel=rbf; total time=   1.6s
[CV] END ..C=1, class_weight=balanced, gamma=0.1, kernel=rbf; total time=   1.6s
[CV] END .C=1, class_weight=balanced, gamma=0.01, kernel=rbf; total time=   1.5s
[CV] END .C=1, class_weight=balanced, gamma=0.0

In [25]:
print(classification_report(Y_test, grid_pred_lin))
print(classification_report(Y_test_rus, grid_pred_rus))
print(classification_report(Y_test_ros, grid_pred_ros))

              precision    recall  f1-score   support

           0       0.80      1.00      0.89      2414
           1       0.00      0.00      0.00       586

    accuracy                           0.80      3000
   macro avg       0.40      0.50      0.45      3000
weighted avg       0.65      0.80      0.72      3000

              precision    recall  f1-score   support

           0       0.49      0.99      0.66       605
           1       0.44      0.01      0.01       618

    accuracy                           0.49      1223
   macro avg       0.47      0.50      0.34      1223
weighted avg       0.47      0.49      0.33      1223

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      2409
           1       1.00      0.94      0.97      2369

    accuracy                           0.97      4778
   macro avg       0.97      0.97      0.97      4778
weighted avg       0.97      0.97      0.97      4778



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [26]:
churn=df.loc[df["Exited"]==1]

In [27]:
not_churn =df.loc[df["Exited"]==0]

In [28]:
not_churn["Tenure"].value_counts().sort_values()

,count
Tenure,
0,318
10,389
9,771
6,771
4,786
3,796
5,803
1,803
8,828


In [29]:
churn["Tenure"].value_counts().sort_values()

,count
Tenure,
0,95
10,101
7,177
6,196
8,197
2,201
4,203
5,209
3,213


In [30]:
not_churn["NumOfProducts"].value_counts()

,count
NumOfProducts,
1,4288
0,3675


In [31]:
churn["NumOfProducts"].value_counts()

,count
NumOfProducts,
0,1409
1,628


In [32]:
churn["IsActiveMember"].value_counts()

,count
IsActiveMember,
0,1302
1,735


In [33]:
churn["CreditScore"].describe([0.05,0.25,0.50])

,CreditScore
count,2037.000000
mean,645.351497
std,100.321503
min,350.000000
5%,479.000000
25%,578.000000
50%,646.000000
max,850.000000
